In [ ]:
# ---------- QUIET MODE (put these lines at very top) ----------
import os, sys, types, warnings
os.environ["TQDM_NOTEBOOK"] = "0"  # tqdm.auto가 노트북 모드로 안 가게

try:
    import tqdm as _tqdm
    _auto = types.ModuleType("tqdm.auto"); _auto.tqdm = _tqdm.tqdm
    sys.modules["tqdm.auto"] = _auto
except Exception:
    pass

from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", message=".*IProgress not found.*", category=UserWarning)
warnings.filterwarnings("ignore", message=".*X does not have valid feature names.*", category=UserWarning)
# -------------------------------------------------------------

import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

from rolling_framework import ExpandingRunner, make_strategy

# ---------------- CONFIG ----------------
DATA_FILE     = os.path.join("", "dataset.csv")     # index='Time'
Y_COLS        = ["xr_2", "xr_3", "xr_5", "xr_7", "xr_10"]
Y_10_COL      = "xr_10"

SLOPE_PREFIX  = "s_"
FWD_PREFIX    = "fwd_"
MACRO_PREFIX  = "F"

PERIOD        = ["197108", "202312"]
BURN_IN_END   = "200609"
HORIZON       = 12
SHOW_PROGRESS = True

# 반복 횟수 및 seed
N_RUNS        = 20
BASE_SEED     = 42

# ---------------- Ridge 설정 (TS-CV) ----------------
# base Ridge 파라미터(고정값). alpha는 CV grid에서 조정.
RIDGE_PARAMS_BASE = dict(
    random_state=None
)

RIDGE_CV = dict(
    mode="tscv",
    n_splits=10,
    grid={
        "alpha": [1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0],
    },
)

# ---------------- MLP 설정 (hold-out CV) ----------------
# Sklearn MLPRegressor용 base 파라미터(공통)
MLP_PARAMS_BASE = dict(
    max_iter=2000,
    early_stopping=True,
    learning_rate_init=1e-3,
    tol=1e-5,
    # hidden_layer_sizes, alpha는 CV grid로 튜닝
    random_state=None,
)

# hold-out CV: 한 번의 train/validation split
MLP_CV = dict(
    mode="holdout",
    val_ratio=0.2,
    grid={
        # SDMlp 노트북에서 쓰던 구조에 맞춰, 비교적 작은 네트워크와 큰 L2
        "hidden_layer_sizes": [(16,), (16, 8)],
        "alpha": [1e3, 1e5, 1e7],
    },
)

# ---------------- SDMLP base params ----------------
# SDMLP 내부 구현에 맞춰 사용 (표준화는 외부에서 처리한다고 가정)
SDMLP_PARAMS_BASE = dict(
    hidden_layer_sizes=(16,),   # 필요시 수정
    alpha=1e5,
    max_iter=2000,
    early_stopping=True,
    learning_rate_init=1e-3,
    tol=1e-5,
    random_state=None,
    # slope_lr 등 추가 파라미터가 있다면 SDMLP 전략 구현에 맞게 확장
)

# ---------------- mini utils ----------------
def read_df(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, index_col="Time")
    df.index = df.index.astype(str)
    return df

def features(df: pd.DataFrame, *, use_fwd: bool, use_macro: bool) -> pd.DataFrame:
    parts = []
    if use_fwd:
        parts.append(df.loc[:, df.columns.str.startswith(FWD_PREFIX)])
    if use_macro:
        parts.append(df.loc[:, df.columns.str.startswith(MACRO_PREFIX)])
    if not parts:
        raise ValueError("No features selected (fwd/macro).")
    return pd.concat(parts, axis=1)

def cs_baseline(runner: ExpandingRunner, df: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:
    """
    Per-maturity Campbell–Shiller OLS baseline:
        xr_j ~ s_j

    Returns:
        DataFrame indexed by runner.test_times, columns=Y_COLS
    """
    rows = []
    for t in runner.test_times:
        tr = [s for s in runner.times if s < t]
        if not tr:
            continue
        row = {}
        for ycol in Y_COLS:
            mat  = ycol.split("_", 1)[1]
            scol = f"{SLOPE_PREFIX}{mat}"
            X_tr = df.loc[tr, [scol]].astype(float)
            y_tr = y.loc[tr, ycol].astype(float).values
            reg  = LinearRegression().fit(X_tr, y_tr)
            x_te = df.loc[[t], [scol]].astype(float)
            row[ycol] = float(reg.predict(x_te)[0])
        rows.append(pd.Series(row, name=t))
    return pd.DataFrame(rows).reindex(index=runner.test_times, columns=Y_COLS)

def mse10_from_runner(runner: ExpandingRunner, y_col: str = Y_10_COL) -> pd.Series:
    """
    xr_10에 대한 시점별 제곱오차 SE_t = (y_true - y_pred)^2 반환.
    """
    Y_true, Y_pred = runner.collect_frames()
    se_10 = (Y_pred[y_col] - Y_true[y_col]) ** 2
    se_10.name = "MSE_10"
    return se_10

def save_mse10_mean(se_list, csv_path: str) -> None:
    """
    여러 run의 SE_t 시리즈 리스트를 받아 run 방향으로 평균낸 MSE_10(t)를 CSV로 저장.
    """
    df = pd.concat(se_list, axis=1)
    mse_mean = df.mean(axis=1)
    mse_mean.name = "MSE_10"
    mse_mean.to_csv(csv_path, header=True)

# ---------------- RUN EXPERIMENTS -------------------
if __name__ == "__main__":
    df = read_df(DATA_FILE)
    y  = df[Y_COLS].copy()

    # slope 컬럼 (CS / SDMLP용)
    slope_cols = [f"{SLOPE_PREFIX}{col.split('_', 1)[1]}" for col in Y_COLS]
    X_slope = df[slope_cols].copy()
    slope_map = {ycol: f"{SLOPE_PREFIX}{ycol.split('_', 1)[1]}" for ycol in Y_COLS}

    # feature set 정의
    feature_sets = {
        "fwd":       dict(use_fwd=True,  use_macro=False),
        "fwd_macro": dict(use_fwd=True,  use_macro=True),
    }

    for tag, fspec in feature_sets.items():
        print(f"\n================ Feature set: {tag} ================")
        X_feat = features(df, use_fwd=fspec["use_fwd"], use_macro=fspec["use_macro"])
        feat_cols = list(X_feat.columns)

        # SDMLP / CSARM용 X (slope + features)
        X_cs = pd.concat([X_slope, X_feat], axis=1)

        # 모델별 SE(t) accumulator
        mse_runs_ridge      = []
        mse_runs_csarm      = []
        mse_runs_mlp        = []
        mse_runs_sdmlp      = []

        for run in range(N_RUNS):
            seed = BASE_SEED + run
            print(f"\n--- {tag} | run {run+1}/{N_RUNS} | seed={seed} ---")

            # ---------- 1) Ridge (TS-CV) ----------
            ridge_params = dict(RIDGE_PARAMS_BASE)
            ridge_params["random_state"] = seed  # 비록 Ridge는 deterministic이지만 일관성 유지

            ridge = make_strategy(
                "Ridge",
                target_cols=Y_COLS,
                feature_cols=feat_cols,
                params=ridge_params,
                scale=True,
                cv=RIDGE_CV,
            )

            runner_ridge = ExpandingRunner(
                X=X_feat,
                y=y,
                strategy=ridge,
                period=PERIOD,
                burn_in_end=BURN_IN_END,
                horizon=HORIZON,
            )
            runner_ridge.fit_walk(progress=SHOW_PROGRESS, desc=f"Ridge [{tag}] (run={run+1})")

            se_ridge = mse10_from_runner(runner_ridge)
            mse_runs_ridge.append(se_ridge)

            # 첫 run에서만 CS baseline 및 R^2 출력, MAT 저장
            if run == 0:
                y_cs_hat = cs_baseline(runner_ridge, df, y)

                print(f"\n=== Ridge [{tag}] (run=1) ===")
                print("R2OOS vs naive:\n",    runner_ridge.R2OOS(baseline="naive").round(4))
                print("R2OOS vs condmean:\n", runner_ridge.R2OOS(baseline="condmean").round(4))
                print("R2OOS vs CS OLS:\n",   runner_ridge.R2OOS(baseline="custom", benchmark=y_cs_hat).round(4))

                runner_ridge.to_mat(f"ridge_{tag}.mat", baseline="custom", benchmark=y_cs_hat)

            # ---------- 2) CSARM (residual = Ridge, TS-CV) ----------
            csarm = make_strategy(
                "CSARM",
                target_cols=Y_COLS,
                slope_map=slope_map,
                feature_cols=feat_cols,          # residual ridge가 보는 feature
                residual_kind="ridge",
                residual_params=dict(alpha=1.0, random_state=seed),  # alpha는 res_cv에서 override
                scale_res=True,
                res_cv=RIDGE_CV,
            )

            runner_csarm = ExpandingRunner(
                X=X_cs,     # slope + features
                y=y,
                strategy=csarm,
                period=PERIOD,
                burn_in_end=BURN_IN_END,
                horizon=HORIZON,
            )
            runner_csarm.fit_walk(progress=SHOW_PROGRESS, desc=f"CSARM(ridge) [{tag}] (run={run+1})")

            se_csarm = mse10_from_runner(runner_csarm)
            mse_runs_csarm.append(se_csarm)

            if run == 0:
                print(f"\n=== CSARM(ridge) [{tag}] (run=1) ===")
                print("R2OOS vs naive:\n",    runner_csarm.R2OOS(baseline="naive").round(4))
                print("R2OOS vs condmean:\n", runner_csarm.R2OOS(baseline="condmean").round(4))
                print("R2OOS vs CS OLS:\n",   runner_csarm.R2OOS(baseline="custom", benchmark=y_cs_hat).round(4))

                runner_csarm.to_mat(f"csarm_ridge_{tag}.mat", baseline="custom", benchmark=y_cs_hat)

            # ---------- 3) Plain MLP (hold-out CV) ----------
            mlp_params = dict(MLP_PARAMS_BASE)
            mlp_params["random_state"] = seed

            mlp = make_strategy(
                "MLP",
                target_cols=Y_COLS,
                feature_cols=feat_cols,
                params=mlp_params,
                # 표준화는 외부에서 처리한다고 가정
                scale=False,
                cv=MLP_CV,
            )

            runner_mlp = ExpandingRunner(
                X=X_feat,
                y=y,
                strategy=mlp,
                period=PERIOD,
                burn_in_end=BURN_IN_END,
                horizon=HORIZON,
            )
            runner_mlp.fit_walk(progress=SHOW_PROGRESS, desc=f"Plain MLP [{tag}] (run={run+1})")

            se_mlp = mse10_from_runner(runner_mlp)
            mse_runs_mlp.append(se_mlp)

            if run == 0:
                print(f"\n=== Plain MLP [{tag}] (run=1) ===")
                print("R2OOS vs naive:\n",    runner_mlp.R2OOS(baseline="naive").round(4))
                print("R2OOS vs condmean:\n", runner_mlp.R2OOS(baseline="condmean").round(4))
                print("R2OOS vs CS OLS:\n",   runner_mlp.R2OOS(baseline="custom", benchmark=y_cs_hat).round(4))

                runner_mlp.to_mat(f"mlp_{tag}.mat", baseline="custom", benchmark=y_cs_hat)

            # ---------- 4) SDMLP (구현에 맞게 사용, 여기서는 CV 미적용) ----------
            sdmlp_params = dict(SDMLP_PARAMS_BASE)
            sdmlp_params["random_state"] = seed

            # SDMLP 전략 구현에서 요구하는 인자에 맞춰 사용
            sdmlp = make_strategy(
                "SDMLP",
                target_cols=Y_COLS,
                slope_map=slope_map,
                feature_cols=feat_cols,   # residual MLP는 fwd/macro 사용
                params=sdmlp_params,
            )

            runner_sdmlp = ExpandingRunner(
                X=X_cs,   # slope + features
                y=y,
                strategy=sdmlp,
                period=PERIOD,
                burn_in_end=BURN_IN_END,
                horizon=HORIZON,
            )
            runner_sdmlp.fit_walk(progress=SHOW_PROGRESS, desc=f"SDMLP [{tag}] (run={run+1})")

            se_sdmlp = mse10_from_runner(runner_sdmlp)
            mse_runs_sdmlp.append(se_sdmlp)

            if run == 0:
                print(f"\n=== SDMLP [{tag}] (run=1) ===")
                print("R2OOS vs naive:\n",    runner_sdmlp.R2OOS(baseline="naive").round(4))
                print("R2OOS vs condmean:\n", runner_sdmlp.R2OOS(baseline="condmean").round(4))
                print("R2OOS vs CS OLS:\n",   runner_sdmlp.R2OOS(baseline="custom", benchmark=y_cs_hat).round(4))

                runner_sdmlp.to_mat(f"sdmlp_{tag}.mat", baseline="custom", benchmark=y_cs_hat)

        # ------ run 방향 평균 MSE_10(t) 저장 ------
        save_mse10_mean(mse_runs_ridge, f"ridge_{tag}_mse10_avg.csv")
        save_mse10_mean(mse_runs_csarm, f"csarm_ridge_{tag}_mse10_avg.csv")
        save_mse10_mean(mse_runs_mlp,   f"mlp_{tag}_mse10_avg.csv")
        save_mse10_mean(mse_runs_sdmlp, f"sdmlp_{tag}_mse10_avg.csv")

        print(f"\n>>> [{tag}] 평균 MSE_10(t) CSV 저장 완료.")